In [1]:
#| default_exp merging
import sys
from pathlib import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

# Mesclagem
> Funções auxiliares para mesclar e limpar as várias fontes de dados

In [2]:
#| export
from itertools import product
import pandas as pd
from geopy.distance import geodesic

from extracao.reading import read_aero, read_base

MAX_DIST = 10  # Km
COLS = ["Frequency", "Latitude", "Longitude", "Description"]

### Aeronáutica
Funções auxiliares para mesclar registros que são iguais das diversas bases da aeronáutica, i.e. estão a uma distância menor que `DIST` e verificar a validade da mesclagem

In [3]:
#| export
def check_add_row(
    df,  # DataFrame para mesclar adicionar o registro
    f,  # Frequência (MHz) em análise do registro
    rows,  # Lista de registros para mesclar
    dicts,  # Dicionário fonte dos registros
) -> pd.DataFrame:  # DataFrame com o registro adicionado
    """Mescla os registros em `rows` de frequência `f` e os adiciona como uma linha do DataFrame `df`
    Os registros em `rows` somente são mesclados se ainda constarem nos dicionários fonte `dicts`
    Após a mesclagem, os registros são removidos dos dicionários fonte `dicts`
    """
    if all(row.Index in dict for row, dict in zip(rows, dicts)):
        lat = sum(row.Latitude for row in rows) / len(rows)
        long = sum(row.Longitude for row in rows) / len(rows)
        desc = " | ".join(row.Description for row in rows)
        d = {"Frequency": f, "Latitude": lat, "Longitude": long, "Description": desc}
        for row, reg in zip(rows.copy(), dicts.copy()):
            reg.pop(row.Index)
        return pd.concat([df, pd.DataFrame(d, index=[0])], ignore_index=True)
    return df


In [4]:
#| export
def get_subsets(
    f,  # Frequência (MHz) em análise do registro
    *dfs,  # Conjunto de DataFrames a serem analisados
) -> list:  # Lista com subconjuntos de registros de frequência `f` para cada df em `dfs`
    """Retorna os subconjuntos de registros de frequência `f` em cada dataframe `dfs`
    Os subconjuntos são retornados em forma de dicionário, onde a chave é o índice do registro
    """
    return [
        {s.Index: s for s in df.loc[df.Frequency == f, COLS].itertuples()} for df in dfs
    ]


In [5]:
icao, aisw, aisg, redemet = read_aero(Path.cwd().parent / 'dados', True)

/home/melinda/micromamba/envs/db/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
for c in icao.columns:
    icao[c] = icao[c].astype('string')
for c in aisw.columns:
    aisw[c] = aisw[c].astype('string')

In [8]:
df = icao.copy()
for left in icao.itertuples():
    for right in aisw[aisw.Frequency == left.Frequency].itertuples():
        if (
            geodesic(
                   (left.Latitude, left.Longitude), (right.Latitude, right.Longitude)
                ).km
                <= MAX_DIST
            ):
                df.loc[left.Index, 'Description'] = left.Description + ' | ' + right.Description
                aisw.drop(right.Index)
                break

In [9]:
aisw

,Frequency,Latitude,Longitude,Description,Service,Station
0,0.2,-22.9233341217041,-42.07149887084961,"[AISW] SBCB-RDONAV, NDB BFR, Cabo Frio",-1,-1
1,0.205,-1.4863333702087402,-56.39783477783203,"[AISW] SBTB-RDONAV, NDB PTT, Trombetas",108,684424649
2,0.205,-5.386166572570801,-35.53099822998047,"[AISW] SNXX-RDONAV, NDB MXN, Maxaranguape",-1,-1
3,0.205,-5.386166572570801,-35.53099822998047,"[AISW] SBNT-RDONAV, NDB MXN, CAMPO AUGUSTO SEVERO",-1,-1
4,0.21,-19.56100082397461,-46.96466827392578,"[AISW] SBAX-RDONAV, NDB ARX, Romeu Zema",-1,-1
...,...,...,...,...,...,...
939,1203.0,-8.136500358581543,-34.92733383178711,"[DOC] SBRF-RDONAV, VOR/DME REC, Guararapes - G...",-1,-1
940,1204.0,-19.68899917602539,-47.06050109863281,"[DOC] SBAX-RDONAV, VOR/DME ARX, Romeu Zema (Gr...",-1,-1
941,1207.0,-1.3843333721160889,-48.47850036621094,"[DOC] SBBE-RDONAV, VOR/DME BEL, Val de Cans - ...",-1,-1
942,1209.0,-15.86501407623291,-47.90018844604492,"[DOC] SBBR-RDONAV, VOR/DME VJK, Presidente Jus...",-1,-1


In [ ]:
#| export
def merge_closer(
    frequencies,  # Lista de frequências em comum
    df,  # DataFrame de saída
    df_left,  # DataFrame 1 de entrada da esquerda
    df_right,  # DataFrame 2 de entrada da direita
) -> pd.DataFrame:  # DataFrame de saída com as frequências em comum de `df_left` e `df_right` mescladas ou inseridas individualmente
    """Mescla os registros de frequência `frequencies` de `df_left` e `df_right` em `df`
    Essa função é utilizada para mesclar registros que possuem frequências em comum listadas em `frequencies`
    Os registros são mesclados se a distância entre eles for menor que `MAX_DIST`
    do contrário são adicionados individualmente como uma linha no DataFrame de saída `df`
    """
    for f in frequencies:
        sa, sb = get_subsets(f, df_left, df_right)
        if all(
            [sa, sb]
        ):  # Somente há registros para mesclar se estiverem nos dois conjuntos
            for fa, fb in list(product(sa.copy().values(), sb.copy().values())):
                if (
                    geodesic(
                        (fa.Latitude, fa.Longitude), (fb.Latitude, fb.Longitude)
                    ).km
                    <= MAX_DIST
                ):
                    df = check_add_row(df, f, [fa, fb], [sa, sb])
        for reg in [
            sa,
            sb,
        ]:  # Do contrário os registros são adicionados individualmente ao DataFrame
            for r in reg.copy().values():
                df = check_add_row(df, f, [r], [reg])
    return df


In [ ]:
#| hide
# def merge_single(frequencies, # Lista de frequências em comum
#                  df, # DataFrame de saída
#                  df_left # DataFrame de entrada
# )->pd.DataFrame: # DataFrame de saída com as frequências `frequencies` de `df_left` mescladas ou inseridas individualmente em `df`
#     """Mescla os registros de frequência `frequencies` de `df_left` em `df`"""
#     for f in frequencies:
#         if sa := get_subsets(f, df_left)[0]:
#             for fa in sa.copy().values():
#                 df = check_add_row(df, f, [fa], [sa])
#     return df

In [ ]:
#| export
def merge_triple(
    frequencies,  # Lista de frequências em comum
    df,  # DataFrame de saída
    df_left,  # DataFrame 1 de entrada
    df_middle,  # DataFrame 2 de entrada
    df_right,  # DataFrame 3 de entrada
) -> pd.DataFrame:  # DataFrame de saída com as frequências `frequencies` de `df_left`, `df_middle` e `df_right` mescladas ou inseridas individualmente em `df`
    """Mescla os registros de frequência `frequencies` de `df_left`, `df_middle` e `df_right` em `df`
    Essa função é utilizada para mesclar registros que possuem frequências em comum listadas em `frequencies`
    Os registros são mesclados se a distância entre eles for menor que `MAX_DIST`
    do contrário são adicionados individualmente como uma linha no DataFrame de saída `df`
    """
    for f in frequencies:
        sa, sb, sc = get_subsets(f, df_left, df_middle, df_right)
        if all([sa, sb, sc]):
            for fa, fb, fc in list(
                product(sa.copy().values(), sb.copy().values(), sc.copy().values())
            ):
                dab = geodesic(
                    (fa.Latitude, fa.Longitude), (fb.Latitude, fb.Longitude)
                ).km
                dac = geodesic(
                    (fa.Latitude, fa.Longitude), (fc.Latitude, fc.Longitude)
                ).km
                dbc = geodesic(
                    (fb.Latitude, fb.Longitude), (fc.Latitude, fc.Longitude)
                ).km
                if all(d <= MAX_DIST for d in [dab, dac, dbc]):
                    df = check_add_row(df, f, [fa, fb, fc], [sa, sb, sc])
                elif all(d > MAX_DIST for d in [dac, dbc]):
                    df = check_add_row(df, f, [fa, fb], [sa, sb])
                elif all(d > MAX_DIST for d in [dab, dac]):
                    df = check_add_row(df, f, [fa, fc], [sb, sc])
                elif all(d > MAX_DIST for d in [dab, dbc]):
                    df = check_add_row(df, f, [fa, fc], [sa, sc])
        for reg in [sa, sb, sc]:
            for r in reg.copy().values():
                df = check_add_row(df, f, [r], [reg])
    return df


In [ ]:
#| export
def check_merging(
    df,  # Dataframe to check
    freqs,  # Sets of Frequencies
    dfs,  # Dataframes sources of `df`
):  # True se a mesclagem dos registros de `icao`, `aisw` e `aisg` no DataFrame `df` está consistente
    """Verifica a validade da mesclagem dos registros de `icao`, `aisw` e `aisg` em `df`"""
    assert sum(df[df.Frequency.isin(f)].shape[0] for f in freqs) == len(
        df
    ), "Número de registros inconsistente"
    single_sum = df.Description.str.split("\|").apply(lambda x: len(x)).sum()
    assert single_sum == sum(len(d) for d in dfs), "Número de registros inconsistente"


In [ ]:
#| export
def get_frequencies_set(
    df1: pd.DataFrame,  # DataFrame 1
    df2: pd.DataFrame,  # DataFrame 2
    df3: pd.DataFrame,  # DataFrame 3
) -> tuple:  # Tupla com todos os subconjuntos do Diagrama de Venn das frequências de `df1`, `df2` e `df3`
    """Retorna todos os conjuntos de frequências do Diagrama de Venn entre os registros de `df1`, `df2` e `df3`"""
    f1 = set(df1.Frequency.tolist())
    f2 = set(df2.Frequency.tolist())
    f3 = set(df3.Frequency.tolist())
    ABC = f1.intersection(f2).intersection(f3)
    AB = f1.intersection(f2).difference(ABC)
    BC = f2.intersection(f3).difference(ABC)
    AC = f1.intersection(f3).difference(ABC)
    A = f1.difference(f2).difference(f3)
    B = f2.difference(f3).difference(f1)
    C = f3.difference(f1).difference(f2)
    return A, B, C, AB, AC, BC, ABC


In [ ]:
#| export
def merge_aero(
    folder,  # Pasta onde estão os arquivos de entrada
) -> pd.DataFrame:  # DataFrame com as bases da Aeronáutica com registros mesclados
    """Mescla os registros de mesma frequência e próximos dos arquivos da aeronáutica em `folder`"""
    icao = read_icao(folder).loc[:, COLS].drop_duplicates().reset_index(drop=True)
    aisw = read_aisw(folder).loc[:, COLS].drop_duplicates().reset_index(drop=True)
    aisg = read_aisg(folder).loc[:, COLS].drop_duplicates().reset_index(drop=True)
    df = pd.DataFrame(columns=COLS)
    A, B, C, AB, AC, BC, ABC = get_frequencies_set(icao, aisw, aisg)
    df = merge_closer(AB, df, icao, aisw)
    df = merge_closer(AC, df, icao, aisg)
    df = merge_closer(BC, df, aisw, aisg)
    df = merge_triple(ABC, df, icao, aisw, aisg)
    df = pd.concat(
        [
            df,
            icao[icao.Frequency.isin(A)],
            aisw[aisw.Frequency.isin(B)],
            aisg[aisg.Frequency.isin(C)],
        ]
    )
    df = df.drop_duplicates().sort_values("Frequency").reset_index(drop=True)
    check_merging(df, [A, B, C, AB, AC, BC, ABC], [icao, aisw, aisg])
    return df

In [ ]:
aero = merge_aero(Path.cwd().parent / 'dados')

In [ ]:
aero.to_parquet(Path.cwd().parent / 'dados' / 'aero.parquet.gzip', compression='gzip')